In [4]:
%%time
# 이미지 스플릿 하기
# https://inhovation97.tistory.com/36

import os
import glob
import math
import shutil
import random

# 원본 이미지 폴더속 이미지 개수
doc_img = sorted(glob.glob('D:/coding/data_set/lomin/train/'+'/*'))
print('doc_img 폴더 이미지 개수 : {}'.format(len(doc_img)))

# 나눌 이미지 수
doc_split = round(len(doc_img)*0.1)
print('doc_split : split 이미지 수 : {}'.format(doc_split))

# 이미지 나누기 정의
def split(img_list, test_count, train_path, test_path):

    test_files=[]
    for i in random.sample( img_list, test_count ):
        test_files.append(i)
    
    # 차집합으로 train/test 리스트 생성하기
    train_files = [x for x in img_list if x not in test_files]

    for k in train_files:
        shutil.copy(k, train_path)

    for c in test_files:
        shutil.copy(c, test_path)

    print('train 폴더 이미지 개수 : {}\n test 폴더 이미지 개수 : {}'.format(len(glob.glob(train_path+'/*')), len(glob.glob(test_path+'/*'))))

doc_img 폴더 이미지 개수 : 11948
doc_split : split 이미지 수 : 1195
Wall time: 51 ms


In [5]:
# 이미지 나누기 (각 지정 폴더로 들어감)
tr_path = 'D:/coding/data_set/lomin/train_copy2/train'
te_path = 'D:/coding/data_set/lomin/train_copy2/test'

split(doc_img, doc_split, tr_path, te_path)

train 폴더 이미지 개수 : 10753
 test 폴더 이미지 개수 : 1195


---

# 데이터 전처리

### 신규 이미지는 xml 을 다시 만들어 줘야 함

In [1]:
# Annotation 새로 만들기 (한개만)
#https://junyoung-jamong.github.io/machine/learning/2019/01/23/%EB%B0%94%EC%9A%B4%EB%94%A9%EB%B0%95%EC%8A%A4%EB%A5%BC-%ED%8F%AC%ED%95%A8%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%A6%9D%ED%8F%AD%EC%8B%9C%ED%82%A4%EA%B8%B0-with-imgaug.html

# 바운딩박스 복붙 (Annotation정보 읽기)
import os
import glob
import cv2
import xml.etree.ElementTree as ET

# Annotation 반복 만들기 (기본 0도)
def annotation_one_xml(origin_path, make_path):
    
    # 폴더 내 이미지 리스트
    # img_list = sorted(glob.glob(origin_path + '/*'))

    # for i in img_list:
        # File = open(origin_path + i.split('\\')[1].split('.')[0] + ".xml", "w")
        File = open(origin_path + make_path.split('/')[-1].split('.')[0] + ".xml", "w")
        print("""<annotation>
    <folder>test_after</folder>
    <filename>{}.png</filename>
    <path>D:\coding\data_set\lomin\\train_copy\\train\\{}.png</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>1654</width>
        <height>2339</height>
        <depth>1</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>0</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>355</xmin>
            <ymin>503</ymin>
            <xmax>1299</xmax>
            <ymax>1836</ymax>
        </bndbox>
    </object>
</annotation>
        """.format(make_path.split('/')[-1].split('.')[0], make_path.split('/')[-1].split('.')[0]), file=File)

        File.close()

---

# 이미지 크롭 (랜덤하게 부분 돌면서)

In [ ]:
## 이미지 크롭 (500*500)

import os
import glob
import cv2
import random

# 경로 설정
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

def crop_500(input_path, output_path):
    img_list = sorted(glob.glob(input_path + '/*'))

    for i in img_list:

        src = cv2.imread(i, cv2.IMREAD_COLOR)
        
        height, width, channel = src.shape
        
        ## 이미지 에서 500*500 이미지를 crop 함
        x_range = width - 501
        y_range = height - 501
        
        for j in range(0, 3):
            x1 = x_random = random.randint(0, x_range)
            y1 = y_random = random.randint(0, y_range)
            x2 = x1 + 500
            y2 = y1 + 500 
            # print(y1, y2, x1, x2)
            
            dst = src[y1:y2, x1:x2]
            
            cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '(' + str(j) + ')' + '.png', dst)
            
            # 문제점 - 회전을 한 이미지 crop 시 좌표도 회전하게 됨. (이미지 짤림)
            # affine 은 이미지를 회전, 왜곡 해도 좌표 정보를 그대로 가지고 있음.

---

# 여백 흰색으로 이미지 회전

In [1]:
# 이미지 회전시키기 (원본)

import os
import glob
import cv2

# 경로 설정
# input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
# output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

input_path = 'D:/coding/data_set/lomin/train_copy3/temp1/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp2/'

def ro_img_lite(angle, input_path, output_path):
    # 경로 폴더 만들기
    try :
        os.makedirs(output_path)
    except:
        pass

    img_list = sorted(glob.glob(input_path + '/*'))

    for i in img_list:
        if not '.ini' in i:
            # print(i.split('\\')[1].split('.')[0])
            src = cv2.imread(i, cv2.IMREAD_COLOR)
            height, width, channel = src.shape
            my_angle = angle

            ##                                  중심점(center), 각도(angle), 비율(scale)
            matrix = cv2.getRotationMatrix2D((width/2, height/2), -my_angle, 1)
            # 비율 : 가로길이 / 대각선

            #                                           흰색 : borderValue=(255, 255, 255)
            dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

            # 변경 이미지 저장 (파일명_각도)
            # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
            cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '.png', dst)



In [4]:
## 이미지 크롭 (500*500)

import os
import glob
import cv2
import random

# 경로 설정
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

def img_crop_ro(my_angle, input_path, output_path):
    img_list = sorted(glob.glob(input_path + '/*'))

    for i in img_list:

        src = cv2.imread(i, cv2.IMREAD_COLOR)
        
        height, width, channel = src.shape
        
        ## 이미지 에서 500*500 이미지를 crop 함
        x_range = width - 501
        y_range = height - 501
        
        for j in range(0, 3):
            x1 = x_random = random.randint(0, x_range)
            y1 = y_random = random.randint(0, y_range)
            x2 = x1 + 500
            y2 = y1 + 500 
            # print(y1, y2, x1, x2)
            
            dst = src[y1:y2, x1:x2]


            cv2.imwrite(output_path + i.split('\\')[1].split('.')[0]  + '(' + str(j) + ')' + '_' + str(my_angle) + '.png', dst)
            

        ro_img_lite(my_angle, output_path, output_path)
            # 문제점 - 회전을 한 이미지 crop 시 좌표도 회전하게 됨. (이미지 짤림)
            # affine 은 이미지를 회전, 왜곡 해도 좌표 정보를 그대로 가지고 있음.

In [5]:
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

img_crop_ro(66, input_path, output_path)

---

# 이미지 크롭하고 회전시켜 저장하기

In [ ]:
def rotation_crop(angle, path):
    # print(i.split('\\')[1].split('.')[0])
    src = cv2.imread(path, cv2.IMREAD_COLOR)
    height, width, channel = src.shape
    my_angle = angle

    ##                                  중심점(center), 각도(angle), 비율(scale)
    matrix = cv2.getRotationMatrix2D((width/2, height/2), -my_angle, 1)
    # 비율 : 가로길이 / 대각선

    #                                           흰색 : borderValue=(255, 255, 255)
    dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

    # 변경 이미지 저장 (파일명_각도)
    # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
    cv2.imwrite(path, dst)

In [ ]:
import os
import glob
import cv2
import random
import numpy as np

def lotation_img(img_count, input_path, output_path):
    # 폴더 만들기
    try :
        os.makedirs(output_path)
    except:
        pass

    img_list = sorted(glob.glob(input_path + '/*'))

    # 각도
    for j in range(0, 360):
        # 전체 이미지 중 랜덤하게 추출
        random_list = np.random.choice(img_list, img_count, replace=False)
        
        for i in random_list:
            src = cv2.imread(i, cv2.IMREAD_COLOR)
            
            height, width, channel = src.shape
            
            ## 이미지 에서 500*500 이미지를 crop 함
            x_range = width - 501
            y_range = height - 501
            
            for k in range(0, 3):
                x1 = x_random = random.randint(0, x_range)
                y1 = y_random = random.randint(0, y_range)
                x2 = x1 + 500
                y2 = y1 + 500 
                # print(y1, y2, x1, x2)
                
                dst = src[y1:y2, x1:x2]


                total_path = output_path + i.split('\\')[-1].split('.')[0] + '('+str(k)+")_" + str(j) + '.png' 
                cv2.imwrite(total_path, dst)
                
                # 이미지 회전
                rotation_crop(j, total_path)
            


In [ ]:
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

lotation_img(30, input_path, output_path)

---

In [2]:
## Annotation 읽기

# https://junyoung-jamong.github.io/machine/learning/2019/01/23/%EB%B0%94%EC%9A%B4%EB%94%A9%EB%B0%95%EC%8A%A4%EB%A5%BC-%ED%8F%AC%ED%95%A8%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%A6%9D%ED%8F%AD%EC%8B%9C%ED%82%A4%EA%B8%B0-with-imgaug.html

import xml.etree.ElementTree as ET

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [3]:
## 이미지 데이터 읽기 (파일 1개 읽기)

from os import listdir
import cv2
import numpy as np

def read_one_train_dataset(dir, file):
    images = []
    annotations = []

    if 'jpg' in file.lower() or 'png' in file.lower():
        images.append(cv2.imread(dir + file, 1))
        annotation_file = file.replace(file.split('.')[-1], 'xml')
        bounding_box_list, file_name = read_anntation(dir + annotation_file)
        annotations.append((bounding_box_list, annotation_file, file_name))

    # images = np.array(images)

    return images, annotations

---

In [25]:
## 폴더 생성 + 검은 배경 추가 + 360도 회전 + annotation 추가 (회전각에 대한 박스 변경)

import os
import glob
import cv2
import imgaug as ia
from imgaug import augmenters as iaa
from pascal_voc_writer import Writer
import xml.etree.ElementTree as ET

# input_path = 'D:/coding/data_set/lomin/train_copy2/train' 
# output_path = 'D:/coding/data_set/lomin/train_copy2/train_angle/'

# img_count = 몇장 씩 => 360도 
def lotation_img(img_count, input_path, output_path):
    
    # 폴더 만들기
    os.makedirs(output_path)

    img_list = sorted(glob.glob(input_path + '/*'))

    for j in range(0, 360):
        # 전체 이미지 중 랜덤하게 추출
        random_list = np.random.choice(img_list, img_count, replace=False)
        
        for i in random_list:
            # print(i.split('\\')[1].split('.')[0])
            src = cv2.imread(i, cv2.IMREAD_COLOR)
            height, width, channel = src.shape
            
            ##                                  중심점(center), 각도(angle), 비율(scale)
            matrix = cv2.getRotationMatrix2D((width/2, height/2), -j, 0.57)
            # 비율 : 가로길이 / 대각선

            #                                           흰색 : borderValue=(255, 255, 255)
            dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

            # 변경 이미지 저장 (파일명_각도)
            fname_angle = i.split('\\')[1].split('.')[0] + '_' + str(j) + '.png'
            cv2.imwrite(output_path + fname_angle, dst)
            
            # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '.png', dst)
            
            # annotation 파일 추가 (기본 0도 파일만 생성)
            xml_path = output_path + fname_angle
            annotation_one_xml(output_path, xml_path)
            
            # 생성된 이미지 기반으로 이미지정보, xml 정보를 읽음
            images, annotations = read_one_train_dataset(output_path, fname_angle)

            image = images[0]
            boxes = annotations[0][0]

            # 추출된 xml 정보 중 bbox 좌표만 추출
            ia_bounding_boxes = []
            for box in boxes:
                ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

            bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)
            
            # 이미지 변환 각종 옵션들 (현재는 회전만 함 - 사용시 아래의 이미지 생성을 사용해야 함)
            seq = iaa.Sequential([
                # iaa.Multiply((1.2, 1.5)),
                iaa.Affine(
                    # translate_px={"x": 40, "y": 60},
                    # scale=(0.5, 0.7),
                    rotate = int(j)
                )
            ])

            seq_det = seq.to_deterministic()

            image_aug = seq_det.augment_images([image])[0]
            bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

            # new_image_file = output_path + annotations[0][2].split('.')[0] + '_' + str(j) + "." + annotations[0][2].split('.')[1]
            new_image_file = output_path + annotations[0][2].split('.')[0] + "." + annotations[0][2].split('.')[1]
            # 이미지 파일 생성 (이미지 파일은 위에서 이미 생성 했음)
            # cv2.imwrite(new_image_file, image_aug)

            h, w = np.shape(image_aug)[0:2]
            voc_writer = Writer(new_image_file, w, h)

            # xml 파일 내부 속성 바꿈
            for i in range(len(bbs_aug.bounding_boxes)):
                bb_box = bbs_aug.bounding_boxes[i]
                # name (label) , x1, x2, y1, y2 값 바꿈
                voc_writer.addObject(j, int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

            # annotation 파일 내부 좌표를 변경함
            voc_writer.save(output_path + annotations[0][1].split('.')[0] + "." + annotations[0][1].split('.')[1])


In [28]:
%%time

## 전처리 실행 (train)

input_path = 'D:/coding/data_set/lomin/train_copy2/train' 
output_path = 'D:/coding/data_set/lomin/train_copy2/train_angle/'

lotation_img(30, input_path, output_path)

Wall time: 21min 59s


In [27]:
%%time

## 전처리 실행 (test)

input_path = 'D:/coding/data_set/lomin/train_copy2/test' 
output_path = 'D:/coding/data_set/lomin/train_copy2/test_angle/'

lotation_img(3, input_path, output_path)

Wall time: 2min 15s


---

In [121]:
from PIL import Image
aaa = Image.open(idx + '000876.png')
imgArray = np.asarray(aaa)
print(imgArray.shape)

(2339, 1654, 3)


---